## 最小角回归（LARS）


   最小角回归是一种变量选择方法，它大大减少了逐步回归过程中的迭代次数，整个求解过程可以被控制在m步内完成.  
   在传统的向前逐步回归中，我们将每一个估计系数初始值设为0，找到和响应变量相关性最大的自变量，然后尽可能地在这个方向上走最远的距离，直到和当前残差值有很大相关性的另一个自变量的出现。最小角回归基于这种思想，它每一次沿着当前所有模型中变量的角平分线走，直到另一个变量的相关性大到足够进入模型.  
   现在具体阐述最小角回归的几何原理和迭代步骤：  
我们的目标是找一个$\hat{\beta}$用于拟合,拟合值$\hat{\mu}=X\hat{\beta},$使得平方误差最小且满足条件$\sum_{j=1}^{m} \hat{\beta_j}<t.$
   首先,引入一些符号表示,设$X$中的变量都是独立的,$X = (x_1,x_2,\cdots,x_m),A$是${1,2,\cdots,m}$的子集,用于记录模型中已有的变量,一开始是空的,$X_A=(\cdots s_jx_j \cdots)_{j\in A},s_j=\pm1,$其中$s_j$由该变量与残差的相关性决定,$G_A =X_A^TX_A,A_A=({1^T}G_A^{-1}{1})^{-1/2},W_A=A_AG_A^{-1}1,$单位角平分线向量$U_A=X_AW_A.$  
    简单给出角平分线向量的求解过程,由单位角平分线的性质$X_A^TU_A=A_A1,||U_A||^2=1,$求解$W_A$和$A_A,X_A^TX_AW_A=A_A1,W_A=(X_A^TX_A)^{-1}A_A1,$代入$U_A^TU_A=1$得$A_A^21^T(X_A^TX_A)^{-1}(X_A^TX_A)(X_A^TX_A)^{-1}1=1,$故$A_A=({1^T}G_A^{-1}{1})^{-1/2}.$  
    算法完整迭代步骤,在计算之前，对$X$进行中心化标准化,对$y$进行中心化操作,拟合值$\mu_A$初始值为$0,$估计系数$\hat{\beta}$初始值为$0,$计算所有变量与当前残差的相关性,即$\hat C=X^Ty,$每一个分量分别代表每一个变量与残差的相关性,找出其中最大的$C_j,$记为$C_{max},$将所有相关性等于$C_{max}$的变量编号放进集合A中，其对应的符号记录在集合$S$中,然后计算当前所有集合A中变量的角平分线,即$X_A$列向量的角平分线,这里需要注意的是之所以引入符号$s_j$是为了是列向量经过符号调整后与残差方向的夹角小于$90^。,$这样才能保证我们前进的方向是正确的,才能使拟合值不断靠近y而不是偏离y,计算角平分线到每一个变量的投影,由于数据经过标准化,也即夹角余弦值,当前角平分线的方向决定我们即将要前进的方向,而要走多远由$\hat\gamma$决定,$\hat\gamma$是使得走完这一步以后更新的残差能够与加入系新的变量以后重新计算出的角平分线平行的数值,即现有A集合中的变量以及加入的新变量与残差的相关性相同,$\hat\gamma=min_{j\in A^c}^+(\frac{C_max-C_j}{A_A-a_j},\frac{C_max+C_j}{A_A+a_j}),$可以看出$\hat\gamma$不仅告诉了我们这一步要走多远,还告诉了我们下一步加入A集的变量有哪些.依次更新$\mu_A,\hat{\beta},\hat C$,其中$\mu_A=\mu_A+\hat\gamma U_A,\hat{\beta} = \hat{\beta}+\hat\gamma \beta^+,\beta^+$为$W_A$填充$0$使维度与$\hat{\beta}$相同,$\hat C = \hat C - \hat\gamma a,$如此迭代下去,m步之内完成求解.
   

In [30]:
mchol <- function(x)
{
  mn <- dim(x)
  m <- mn[1]
  n <- mn[2]
  
  #检验传入参数是否符合要求
  if(m != n) 
  {
    return ("Wrong dimensions of matrix!")
  }
  if(sum(t(x) != x) > 0) 
  {
    return ("Input matrix is not symmetrical!")
  }
  
  L <- matrix(0, m, m)
  
  for(i in 1:m)  #以列为单位求解L
  {
    L[i,i] <- sqrt(x[i,i])
    if(i < m)
    {
      L[(i+1):m,i] <- x[(i+1):m,i]/L[i,i]
      
      #更新矩阵，便于下一次同样方法计算
      TLV <- L[(i+1):m,i]
      TLM <- matrix(TLV, m-i, m-i)
      TLM <- sweep(TLM, 2, TLV, "*")
      x[(i+1):m,(i+1):m] <- x[(i+1):m,(i+1):m] - TLM
    }
  }
  L  
}

mforwardsolve=function(L,b){
  mn=dim(L); m=mn[1]; n=mn[2]
  if(m!=n) return ("Wrong dimensions of matrix L!")
  if(m!=length(b)) return ("Wrong dimensions of matrix L or vector b!") #检查输入参数是否符合要求
  x=rep(0,m)
  for(i in 1:m){
    x[i]=b[i]/L[i,i]
    if(i<m) b[(i+1):m]=b[(i+1):m]-x[i]*L[(i+1):m,i]   #更新右端项，逐步减去已求出的未知量对右端项的贡献   
  }
  x  
}

mbacksolve=function(L,b){
  mn=dim(L); m=mn[1]; n=mn[2]
  if(m!=n) return ("Wrong dimensions of matrix L!")
  if(m!=length(b)) return ("Wrong dimensions of matrix L or vector b!") #检查输入参数是否符合要求
  x=rep(0,m)
  for(i in m:1){
    x[i]=b[i]/L[i,i]
    if(i>1) b[(i-1):1]=b[(i-1):1]-x[i]*L[(i-1):1,i]   #更新右端项   
  }
  x  
}

######################################################以上为一些需要用到的函数


mylars = function(x,y,t){
  eps = 1e-8
  xnames = names(x)
  x = as.matrix(cbind(x))
  nm = dim(x)
  n = nm[1]
  m = nm[2]
  one = rep(1, n)#中心化
  meanx = drop(one %*% x)/n
  x = scale(x, meanx, FALSE)
  mu = mean(y)
  y = drop(y - mu)
  normx = sqrt(drop(one %*% (x^2)))#标准化 
  x = scale(x, FALSE, normx)
  
  xtx = t(x)%*%x
  muA = rep(0,n)#初始化拟合值
  betahat = rep(0,m)#初始化估计系数
  Chat = t(x)%*%y#初始化变量和残差的相关性,因为当前拟合值为0,所以残差就是y
  A = NULL
  S = NULL
  k = 1
    
  while(sum(abs(betahat))< t&&k <= m){
    Cmax = max(Chat)
    SS = sign(Chat)
    if(is.null(A)){
      joinA = Chat >= Cmax - eps
      A = (1:m)[joinA]
      S = SS[joinA]
      LA = mchol(t(scale(t(scale(xtx[A,A], FALSE, S)), FALSE, S)))
    }
    else{
      joinA[A] = FALSE
      joinA[-A] = Chat[-A] >= Cmax - eps#已经在模型里的变量不再重复加入
      for(i in (1:m)[joinA]){#利用原有的矩阵更新cholesky分解矩阵
        sign_i = SS[i]
        Lvec = mforwardsolve(LA, xtx[A, i]*sign_i)
        Lvec_ = sqrt(xtx[i,i] - sum(Lvec*Lvec))
        LA = as.matrix(rbind(cbind(LA,0),c(Lvec,Lvec_)))
        A = c(A, i)#记录加入模型的变量编号
        S = c(S, sign_i)#记录对应变量的符号
      }
    GA1 = mbacksolve(t(LA), mforwardsolve(LA, rep(1, length(A))))#求解GA1
    AA = 1/sqrt(sum(GA1))
    WA = AA*GA1
    uA = scale(x[,A], FALSE, S)%*%WA#当前角平分线
    a = scale(xtx[,A], FALSE, S)%*%WA#各个分量在角平分线的投影
    gammahat = min(ifelse(Chat[-A]>0,(Cmax-Chat[-A])/(AA-a[-A]),(Cmax+Chat[-A])/(AA+a[-A])))#找出最优的步长
    muA = muA + gammahat*uA#更新拟合值
    betaplus = rep(0, m)
    betaplus[A] = WA
    betahat = betahat + gammahat*betaplus#更新系数估计
    Chat = Chat - gammahat*a#更新变量与残差的相关性
    k = k+1
    }
  }
  residuals = y - muA
  names(betahat) = xnames 
  betahat_ = scale(t(betahat), FALSE, normx)#还原估计系数
  RSS = apply(residuals^2, 2, sum)#计算残差平方和

  cat('RSS:', RSS,'  ')
  cat('mu:',mu)  
  betahat_
}

library(ElemStatLearn)
data(prostate)
x = prostate[,1:8]
y = prostate[,9]
mylars(x, y, 1000)
      

RSS: 46.30791   mu: 2.478387

lcavol,lweight,age,lbph,svi,lcp,gleason,pgg45
0.4945025,0.490408,0,0.03525647,0.5537084,0,0,0.001386647
